# Algorithms


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/UoB-HPC/minicombust/blob/main/docs/Algorithms.ipynb)

```{toctree}
```



In [76]:
import numpy as np
from numpy.typing import NDArray
from typing import Final

from minicombust import Cell, Face, Geometry, T
from minicombust.utils import minmax
from minicombust.algorithms  import limit_slope

# SIMPLE with corrector steps


SIMPLE slves the momentum equations for velocity using a guess for pressure field and face mass fluxes, after which mass fluxes are updated with the new velocity values. But these will not satisfy pressure continuity equatins.
Flux correction a pressure correcton field.

[Mostsly from CompFDInINdustrialComnustion]

We use a sequential approach to solving the system of discretised PDEs (and particle tracking), i.e. we solve the governing equations one at a time. Picard iteration for non-linearities.

From Dolfyn source:
```
!     This routine assembles and solves the pressure-correction
!     equation using colocated grid. SIMPLE algorithm with one
!     or more corrector steps (non-orthogonality effects taken
!     into account as described in Sect. 8.8 of the corrected
!     2nd printing.
```

We use co-located pressure and velocity points (i.e. at same locations in grid)

What do we do to avoid chekerboarding? Add Rhie and Chow pressure dissipation term as in 8.8 of [Peric]

# Linear Solvers

Since MiniCombust relies on very large (very distributed) problems, the linear solvers used to solve the systems $A\mathbf{x} = \mathbf{b}$ (with A very sparse) must scale. As a results, we use iterative solvers.

PetSc

In addition, the discretise equations have non-linearities, so we must use several sweeps (updating the source terms and coefficients using current iteration values of $x$)

Unlike with structured grid/stencil problems, $A$ does not have a specific band struture.

Multigrid.

In [ ]:
# Determining whether a particle is in a cell
# From Dolfyn subroutie "ParticleInCell" (particles.f90)
"""
We use the cell face normals (positive out) and a normalised position vector from the particle's position to the face centre.
Calculate dot product with cell face centre.
Result: 0 on face, positive inside, negative outside.

Check for all cell faces whether any one is negative.
"""

def normalise(v: T3) -> T3:
    norm=np.linalg.norm(v, ord=1)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return v/norm


def is_particle_in_cell(particle_coords: T3, local_cell_id) -> bool:
  cell_num_faces = cells.num_faces(local_cell_id)
  for face_id in cells.faces(local_cell_id):
      cell1_id, cell2_id 

logical function ParticleInCell(Xp,ic,ifound,Sloppy)
!========================================================================

   use geometry, only: Nfaces, CFace, Face  

   integer, intent(IN)  :: ic
   real, intent(IN)     :: Xp(3), Sloppy
   integer, intent(OUT) :: ifound

   integer              :: j, k, ip, in
   real                 :: Xf(3), Xn(3)
   logical              :: flag 

   flag   = .true.                          ! assume it is in cell ic
   icnt   = 0
   ifound = -1

   do j=1,NFaces(ic)
     k  = CFace(ic,j)
     ip = Face(k)%cell1
     in = Face(k)%cell2
     if( ip == ic )then 
       Xn =  Face(k)%n 
     else if( in == ic )then
       Xn = -Face(k)%n 
     else
       write(*,*)'Error in particle in cell'
     endif

     Xf = Face(k)%x - Xp                  ! vector from particle to face centre

     call normalise(Xf)
     call normalise(Xn)                   ! normal is not normalised normalize

     dotp  = dot_product( Xf , Xn )       ! dot product

!   if( ic == 338 )write(*,*)'x>',j,k,dotp,sloppy
!   if( ic == 218 )write(*,*)'x2>',j,k,dotp,sloppy
     if( dotp < Sloppy )then
       flag   = .false.                   ! Oops! outside of face! (0.0 is on the face)
       icnt   = icnt + 1
       ifound = k

       !if( ic == 225 )hoek = acos( dotp )*180./3.1415927  
       !if( ic == 225 )write(*,*)'f:',ic,j,dotp,' hoek:', hoek,flag,sloppy  
!   write(*,*)'f:',ic,'=>',j,flag,dotp,acos( dotp )*180./3.1415927,sloppy
     endif

   end do
!   if( flag == .false. )write(*,*)'f:',ic,'=>',flag,k

   !if( icnt == 1 )then
   !  write(*,*)'one face failed'
   !else if( icnt > 1 )then
   !  write(*,*)'multiple faces failed ',icnt
   !  ifound = -1
   !endif

   ParticleInCell = flag

end function ParticleInCell


# Calculating gradients
MiniCombust uses [Gauss's Divergence Theorem](https://en.wikipedia.org/wiki/Divergence_theorem) to calculate the gradients in diffusive flux terms.
The divergence theorem allows us to express the flux of a vector field through the surface in terms of the divergence of the field in the volume enclosed.

$$
\int_V (\nabla \cdot \mathbf{\Phi}) dV = \oint_S \mathbf{\Phi} d\vec{s}
$$

or in terms of discrete faces:

$$
(grad \mathbf{\Phi})_P \approx \frac{1}{V_P}\sum_{j=1}^{n}\Phi_j \vec{s_j}
$$

with $\Phi_j$ the value stored at the centre of face $j$

During gradient calculation, we also perform a deferred correction by adjusting the coordinates by weighting the adjacent cells' contributions using the face's interpolation factor property ($\lambda$). This is described in [Peric](Resources.ipynb/#peric) in §8.6.2 (Approximation of Diffusive fluxes). Note that we limit the number of passes of gradient estimation to 2.

Once the gradient has been calculated, we can apply an appropriate slope limiter, in case of bad geometries,
and MiniCombust only supports the approach in [Venkatakrishnan1993].

## Slope limiter: Venkatakrishnan (1993)
The only slope limiter supported by MiniCombust is the approach in  AIAA-93-0880, _On the accuracy of limiters and convergence to steady state solutions_, V.Venkatakrishnan, 1993.

We test the gradient against the original 'neighbour values' as with the approach defined in Barth and Jespersen [ref], and then 
limit it with:

$$
\phi(y) = \frac{y^2 + 2y}{y^2 + y + 2}
$$

The neighbour values we use are those at the surrounding _nodes_ (rather than cell centres or face centres). This approach
is a compromise between overshoot when using face centre values and undershoot when using cell centre values.



In [80]:
from IPython.display import Code
Code(filename="minicombust/algorithms.py", language="python")

import numba
import numpy as np
from typing import Final
from numpy.typing import NDArray
from .data_structures import Geometry, T
from .utils import minmax

# Adapted from Dolfyn gradients.f90, GradientPhiLimiterVenkatarishnan
# MiniCombust only supports "approach 3" (using nodes instead of face centres for limiter)
def limit_slope(Φ: NDArray[T], dΦdX: NDArray[T], mesh: Final[Geometry]) -> NDArray[T]:
    assert(len(Φ) == mesh.num_cells + mesh.num_boundaries) 
    assert(Φ.ndim == 1) # It's a 1D tensor

    assert(Φ.ndim == 1) # It's a 2D tensor
    assert(dΦdX.shape[0] == Φ.shape[0]) # that's the same length as Φ
    assert(dΦdX.shape[1] == 3) # Its nx3

    cells = Geometry.cells

    for local_cell_id, cell in enumerate(mesh.cells):
      Φ_for_this_cell = Φ[local_cell_id]
      neighbour_Φs = Φ[cells.neighbours_with_offsetted_boundary_ids(local_cell_id)]
      Φ_min, Φ_max = minmax(neighbour_Φs)

      # Deterine the smallest and largest deltas to neighbours for this cell
      Δ_min = Φ_min - Φ_for_this_cell
      Δ_max = Φ_max - Φ_for_this_cell        

      # Determine smallest limiter α considering slope to node coords
      surrounding_vertices_coords = cells.vertex_coords(local_cell_id)
      cell_centre = cells.coords[local_cell_id]
      ds = surrounding_vertices_coords - cell_centre
      Δ_faces = np.dot(dΦdX[local_cell_id, :], ds)

      @numba.jit
      def venkatarishnan_factor(Δ_faces: NDArray[T], Δ_min: T, Δ_max: T):
        α = np.arraylike(Δ_faces)
        min_α = np.iinfo(T).max # i.e. MAX_FLOAT
        for Δ_face, i in enumerate(Δ_faces):
          if np.abs(Δ_face) < 1.e-6:
            r = 1000.0
          elif Δ_face > 0.0:
            r = Δ_max / Δ_face
          else:
            r = Δ_min / Δ_face
          α = (r**2+2.0*r)/(r**2+r+2.0)
          min_α = α if α < min_α else min_α
        return min_α
      
      α = venkatarishnan_factor(Δ_faces, Δ_min, Δ_max)
      dΦdX[local_cell_id,:] *= α
    return dΦdX